# 2.1.3 Prompt Anatomy, Context Windows & Common Pitfalls

## Playground Notebook

This notebook covers three foundational topics that will make you a better prompt engineer:

| Topic | What You'll Learn |
|-------|------------------|
| **Anatomy of a Prompt** | The 5 building blocks of every effective prompt |
| **Context Window Management** | How to work within token limits and manage long conversations |
| **Common Pitfalls** | Mistakes that cause bad outputs — and how to fix them |

---

In [88]:
import json
import time
from IPython.display import display, Markdown, HTML
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# ============================================================
#  CONFIGURATION - Change the model name here if needed
# ============================================================
MODEL = "qwen2.5:1.5b"  # Options: "qwen2.5:1.5b", "llama3.2", "mistral", "gemma2", etc.

llm = ChatOllama(model=MODEL)

---

## 1. Anatomy of an Effective Prompt

Every well-crafted prompt is built from up to **5 components**. You don't always need all five, but knowing them helps you write better prompts every time.

| Component | What It Does | Example |
|-----------|-------------|--------|
| **Task** | The core instruction — what you want the model to do | "Summarize this article" |
| **Context** | Background information the model needs | "This is a customer complaint about late delivery" |
| **Role** | Who the model should be (persona) | "You are a senior data scientist" |
| **Format** | How the output should be structured | "Respond as a numbered list" |
| **Constraints** | Boundaries and rules | "Use 3 sentences max. No jargon." |

```
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502  ROLE:        Who should the model be?       \u2502
\u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524
\u2502  CONTEXT:     What does it need to know?     \u2502
\u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524
\u2502  TASK:        What should it do?             \u2502
\u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524
\u2502  FORMAT:      How should it respond?         \u2502
\u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524
\u2502  CONSTRAINTS: What are the limits?           \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
```

### Experiment 1A: Bare Prompt vs. Well-Structured Prompt

Let's see the dramatic difference between a lazy prompt and one that uses all 5 components.

In [89]:
# ============================================================
#  HELPER FUNCTIONS
# ============================================================

def build_messages(message_dicts):
    """Convert a list of role/content dicts into LangChain message objects."""
    type_map = {
        "system": SystemMessage,
        "user": HumanMessage,
        "assistant": AIMessage,
    }
    return [type_map[m["role"]](content=m["content"]) for m in message_dicts]


def chat(messages, **kwargs):
    """Send messages to the model and return the response text."""
    _llm = ChatOllama(model=MODEL, **kwargs) if kwargs else llm
    lc_messages = build_messages(messages)
    start = time.time()
    response = _llm.invoke(lc_messages)
    elapsed = time.time() - start
    content = response.content
    display(Markdown(content))
    print(f"\n\u23f1\ufe0f {elapsed:.2f}s | {len(content)} chars")
    return content


def show_messages(messages):
    """Pretty-print the message list being sent to the model."""
    colors = {"system": "#e74c3c", "user": "#3498db", "assistant": "#2ecc71"}
    html = ""
    for msg in messages:
        role = msg["role"]
        color = colors.get(role, "#888")
        content_preview = msg["content"][:300] + ("..." if len(msg["content"]) > 300 else "")
        html += (
            f'<div style="margin:6px 0;padding:8px 12px;border-left:4px solid {color};'
            f'background:#1e1e1e;border-radius:4px;">'
            f'<strong style="color:{color};text-transform:uppercase;">{role}</strong>'
            f'<br><span style="color:#ccc;white-space:pre-wrap;">{content_preview}</span></div>'
        )
    display(HTML(html))


print(f"✅ Using model: {MODEL}")

✅ Using model: qwen2.5:1.5b


In [90]:
# --- Bare prompt: just the task, nothing else ---
print("=" * 60)
print("BARE PROMPT (task only)")
print("=" * 60)

bare = [
    {"role": "user", "content": "Write about machine learning."}
]
show_messages(bare)
_ = chat(bare)

BARE PROMPT (task only)


Machine Learning (ML) is a field of artificial intelligence that focuses on developing algorithms and statistical models that allow computers to improve their performance in specific tasks through experience or data exposure without being explicitly programmed. ML can be broadly categorized into three main types: supervised learning, unsupervised learning, and reinforcement learning.

1. Supervised Learning:
   This type of ML involves training a model with labeled input-output pairs (x, y). The goal is to find the relationship between inputs and outputs that can predict an output for new or unseen data. Common applications include image classification, speech recognition, and natural language processing.
   
2. Unsupervised Learning:
   In unsupervised learning, models learn from unlabeled data, aiming to discover patterns, structures, or insights within the data itself without explicitly being taught by examples. Examples of unsupervised ML techniques are clustering (e.g., k-means) and association rule mining in market basket analysis.
   
3. Reinforcement Learning:
   This type focuses on algorithms that enable agents to learn how to make decisions by interacting with their environment and receiving feedback through rewards or penalties. It is widely used in robotics, game playing, and autonomous systems.

Key Techniques in Machine Learning:
- **Neural Networks:** Inspired by the structure and function of biological neural networks, these models are designed to recognize features and patterns.
- **Deep Learning:** A subset of machine learning that uses artificial neural networks with many layers (deep architectures) to model complex data structures.
- **Bayesian Methods:** These methods use probability theory to represent uncertainty in data and can be used for classification, regression, and clustering tasks.
- **Ensemble Methods:** Techniques that combine multiple models or different classifiers/feature selection techniques to improve prediction accuracy.

Challenges in Machine Learning:
- Data Quality: The quality of the training data is crucial. Poor-quality data can lead to overfitting (models performing well on training data but poorly on new, unseen data).
- Interpretability: Many ML models are highly complex and difficult to interpret, which limits their use in critical applications.
- Scalability: As datasets grow larger and more complex, the computational resources required for training models also increase.

Technological Impacts:
- **Automation & Efficiency:** Machine learning can automate many tedious tasks that require human decision-making. For instance, recommendation engines in e-commerce websites suggest products based on user behavior data.
- **Healthcare Applications:** ML is being used to analyze medical images and diagnostics reports to improve healthcare outcomes.
- **Cybersecurity:** ML can detect malicious activities by analyzing network traffic patterns or identifying anomalies in financial transactions.

In conclusion, machine learning holds immense promise for transforming industries through improved efficiency, enhanced decision-making capabilities, and increased automation. However, as with any powerful technology, it also presents challenges that require careful consideration of ethical implications and responsible deployment.


⏱️ 6.94s | 3320 chars


In [91]:
# --- Well-structured prompt: all 5 components ---
print("=" * 60)
print("WELL-STRUCTURED PROMPT (all 5 components)")
print("=" * 60)

structured = [
    {
        "role": "system",
        "content": (
            # ROLE
            "You are a tech blogger who writes for beginners. "
            # CONSTRAINTS
            "Keep explanations simple. Avoid jargon. Use analogies from everyday life."
        )
    },
    {
        "role": "user",
        "content": (
            # CONTEXT
            "I'm writing a blog post for people who have never heard of AI. "
            # TASK
            "Write an introduction paragraph explaining what machine learning is. "
            # FORMAT
            "Use exactly 3 sentences. "
            # CONSTRAINTS
            "Do not use the words 'algorithm' or 'neural network'."
        )
    }
]
show_messages(structured)
_ = chat(structured)

print("\n💡 Compare: The structured prompt gives a focused, audience-appropriate response.")

WELL-STRUCTURED PROMPT (all 5 components)


Imagine you're trying to learn how your favorite ice cream shop makes their delicious sundaes. You start by tasting different flavors and noticing that they sometimes add extra toppings for a special flavor. This is a bit like what machine learning does, but instead of making ice cream, it learns from data to make better choices in things like computer vision or voice recognition. It's kind of like if you had a superpower to instantly know the best way to organize your toys based on how often and which ones get played with the most.


⏱️ 1.32s | 538 chars

💡 Compare: The structured prompt gives a focused, audience-appropriate response.


### Experiment 1B: Adding Components Incrementally

Watch how the output improves as we add each component one by one.

In [92]:
# We'll build up the same prompt step by step
levels = [
    {
        "label": "Task only",
        "system": "You are a helpful assistant.",
        "user": "Explain diabetes."
    },
    {
        "label": "Task + Context",
        "system": "You are a helpful assistant.",
        "user": "I'm a newly diagnosed Type 2 diabetes patient. Explain diabetes."
    },
    {
        "label": "Task + Context + Role",
        "system": "You are a compassionate family doctor speaking to a patient.",
        "user": "I'm a newly diagnosed Type 2 diabetes patient. Explain diabetes."
    },
    {
        "label": "Task + Context + Role + Format",
        "system": "You are a compassionate family doctor speaking to a patient.",
        "user": (
            "I'm a newly diagnosed Type 2 diabetes patient. Explain diabetes. "
            "Structure your response as: 1) What it is, 2) Why it happens, 3) What I can do."
        )
    },
    {
        "label": "All 5: Task + Context + Role + Format + Constraints",
        "system": (
            "You are a compassionate family doctor speaking to a patient. "
            "Use simple language. No medical jargon. Keep it under 150 words."
        ),
        "user": (
            "I'm a newly diagnosed Type 2 diabetes patient. Explain diabetes. "
            "Structure your response as: 1) What it is, 2) Why it happens, 3) What I can do."
        )
    }
]

for level in levels:
    print(f"\n{'=' * 60}")
    print(f"  {level['label']}")
    print(f"{'=' * 60}")
    messages = [
        {"role": "system", "content": level["system"]},
        {"role": "user", "content": level["user"]}
    ]
    show_messages(messages)
    result = chat(messages, temperature=0.3)
    print(f"  [Word count: {len(result.split())}]")


  Task only


Diabetes is a chronic metabolic disorder that affects how the body processes glucose, which is a type of sugar found in food. In people with diabetes, the pancreas does not produce enough insulin or the cells do not respond properly to the insulin produced.

Insulin is a hormone that helps regulate blood sugar levels by facilitating the uptake and storage of glucose into cells for energy use. When there is insufficient insulin production or when cells are resistant to its effects, glucose builds up in the bloodstream, leading to high blood sugar levels (hyperglycemia).

There are two main types of diabetes: type 1 diabetes and type 2 diabetes.

Type 1 diabetes occurs when the immune system mistakenly attacks and destroys the beta cells in the pancreas that produce insulin. This results in an absolute deficiency of insulin production and is typically diagnosed in children or young adults.

Type 2 diabetes, on the other hand, develops when the body becomes resistant to insulin (insulin resistance) despite normal levels of insulin being produced. As a result, glucose cannot enter cells as effectively, leading to high blood sugar levels.

Diabetes can cause serious complications if left untreated, including nerve damage, kidney disease, eye problems, and cardiovascular disease. It is important for people with diabetes to monitor their blood sugar levels regularly, follow a healthy diet, exercise regularly, take medications as prescribed by their healthcare provider, and attend regular check-ups to manage the condition effectively.


⏱️ 3.24s | 1552 chars
  [Word count: 241]

  Task + Context


Diabetes is a chronic metabolic disorder characterized by high blood sugar levels, which can lead to serious health complications if not managed properly.
Type 2 diabetes is the most common form of diabetes and occurs when the body becomes resistant to insulin, a hormone that helps regulate blood sugar levels. As a result, glucose builds up in the bloodstream instead of being used for energy or stored for future use.
The pancreas produces insulin, but over time it may become less effective at producing and using this hormone. This leads to high blood sugar levels and can cause damage to various organs and tissues, including the heart, kidneys, eyes, and nerves.
Type 2 diabetes is often diagnosed in adults, although it can also occur in children and teenagers. The risk factors for developing type 2 diabetes include obesity, family history of the disease, age (older people are more likely to develop the condition), physical inactivity, and certain medical conditions such as high blood pressure or high cholesterol.
Diabetes is a serious health issue that requires careful management through diet, exercise, medication, and regular monitoring. It's important for individuals with diabetes to work closely with their healthcare providers to manage their condition effectively and prevent complications.


⏱️ 3.00s | 1313 chars
  [Word count: 205]

  Task + Context + Role


...


⏱️ 4.33s | 1554 chars
  [Word count: 255]

  Task + Context + Role + Format


Diabetes is a chronic condition that affects how the body processes blood sugar (glucose). It occurs when the pancreas doesn't produce enough insulin or when cells become resistant to insulin's effects. Insulin is like a key that allows glucose from food to enter cells, where it's used for energy.

Type 2 diabetes happens because over time, your body becomes less efficient at using insulin effectively. This means more glucose stays in the bloodstream instead of being absorbed by cells.

What you can do:
1. **Healthy Eating**: Focus on a balanced diet rich in fruits, vegetables, whole grains, and lean protein while limiting sugar, refined carbs, and saturated fats.
2. **Regular Exercise**: Aim for at least 30 minutes of moderate exercise most days of the week to help improve your body's sensitivity to insulin.
3. **Weight Management**: If you're overweight or obese, losing even a small amount can make a big difference in managing diabetes.
4. **Medication and Monitoring**: Depending on your condition, medications might be necessary. Regular check-ups with your doctor are crucial for monitoring blood sugar levels and adjusting treatment as needed.

Remember, managing diabetes is a journey that requires dedication and support from healthcare professionals and loved ones.


⏱️ 2.82s | 1288 chars
  [Word count: 200]

  All 5: Task + Context + Role + Format + Constraints


Diabetes is when your body doesn't make enough of the hormone insulin or can't use the insulin properly. This causes high levels of sugar in your blood. Type 2 diabetes usually starts with being overweight and not exercising much.

You can manage it by eating healthy foods, staying active, losing weight if you're extra heavy, taking medicine as prescribed, and checking your blood sugar often. It's important to control what you eat, exercise regularly, and monitor your blood sugar levels closely. If you have any questions or need help, talk to your doctor.


⏱️ 1.41s | 561 chars
  [Word count: 93]


### Experiment 1C: Structure and Clarity Matter

Even with the same information, **how you organize** the prompt affects the output. Compare a messy prompt vs. a clearly structured one.

In [93]:
# --- Messy: all information crammed together ---
print("=" * 60)
print("MESSY PROMPT (same info, poor structure)")
print("=" * 60)

messy = [
    {"role": "user", "content": (
        "I need you to act like a nutritionist and give me a meal plan "
        "for someone who is vegetarian and wants to lose weight and "
        "make it for one day with breakfast lunch dinner and snacks "
        "and also mention calories for each meal and keep total under 1500 calories "
        "and format it nicely"
    )}
]
show_messages(messy)
_ = chat(messy, temperature=0.3)

MESSY PROMPT (same info, poor structure)


Certainly! Here's a well-balanced vegetarian meal plan that should help you achieve your goals of losing weight while keeping the calorie count below 1500. This plan includes breakfast, lunch, dinner, and snacks for one day.

### **Breakfast:**
- **Oatmeal with Almond Milk and Berries**
  - **Ingredients:** 1 cup rolled oats, 2 cups unsweetened almond milk, 1/2 cup mixed berries (strawberries, blueberries), 1 tablespoon chia seeds.
  - **Calories:** Approximately 300 calories.

### **Snack:**
- **Apple with Peanut Butter**
  - **Ingredients:** 1 medium apple, 2 tablespoons natural peanut butter.
  - **Calories:** Approximately 150 calories.

### **Lunch:**
- **Vegetable Stir-Fry with Brown Rice**
  - **Ingredients:** 1 cup brown rice (cooked), mixed vegetables (bell peppers, carrots, snap peas, broccoli florets), tofu cubes, soy sauce, sesame oil.
  - **Calories:** Approximately 400 calories.

### **Snack:**
- **Greek Yogurt with Almonds and Honey**
  - **Ingredients:** 1 cup plain Greek yogurt, 1/2 cup chopped almonds, 1 tablespoon honey.
  - **Calories:** Approximately 350 calories.

### **Dinner:**
- **Quinoa Salad with Chickpeas and Avocado**
  - **Ingredients:** 1 cup cooked quinoa (about 1/4 cup dry), 1 can chickpeas drained, 1 avocado, cherry tomatoes, cucumber, red onion.
  - **Calories:** Approximately 350 calories.

### **Snack:**
- **Carrot Sticks with Hummus**
  - **Ingredients:** 2 cups carrot sticks, 2 tablespoons hummus.
  - **Calories:** Approximately 100 calories.

This meal plan is designed to be balanced and nutritious. It includes a variety of vegetables, whole grains, lean proteins, healthy fats, and fruits, which should help you achieve your weight loss goals while staying within the calorie limit. Adjustments can always be made based on individual preferences or specific dietary needs.


⏱️ 5.26s | 1839 chars


In [94]:
# --- Clean: same info, well-structured with clear sections ---
print("=" * 60)
print("CLEAN PROMPT (same info, clear structure)")
print("=" * 60)

clean = [
    {
        "role": "system",
        "content": "You are a certified nutritionist. Provide evidence-based meal plans."
    },
    {
        "role": "user",
        "content": """Create a one-day meal plan with the following requirements:

**Patient profile:**
- Vegetarian
- Goal: weight loss

**Meals to include:**
- Breakfast, Lunch, Dinner, 2 Snacks

**Constraints:**
- Total calories: under 1500
- Show calorie count for each meal

**Format:**
- Use a Markdown table with columns: Meal, Items, Calories"""
    }
]
show_messages(clean)
_ = chat(clean, temperature=0.3)

print("\n💡 Notice how the structured prompt gives a more organized, complete response.")

CLEAN PROMPT (same info, clear structure)


Certainly! Below is a one-day vegetarian meal plan that meets the specified requirements of being weight loss-friendly and under 1500 calories. The meals are designed to be balanced in macronutrients (carbohydrates, proteins, and fats) while focusing on vegetables for their high fiber content.

### Breakfast: 
- **Items:** Greek Yogurt with Berries
- **Calories:** 320

| Meal | Items | Calories |
|------|-------|----------|
| Breakfast | Greek Yogurt with Berries | 320 |

**Breakfast:**
- Greek yogurt (1 cup) - Provides protein and probiotics.
- Fresh berries (1/2 cup) - Rich in antioxidants, fiber, and vitamins.

### Lunch:
- **Items:** Quinoa Salad with Avocado
- **Calories:** 450

| Meal | Items | Calories |
|------|-------|----------|
| Lunch | Quinoa Salad with Avocado | 450 |

**Lunch:**
- Quinoa (1/2 cup cooked) - Provides complex carbohydrates and fiber.
- Avocado (1 medium) - High in healthy fats, fiber, and potassium.

### Dinner:
- **Items:** Stir-Fried Vegetables with Tofu
- **Calories:** 300

| Meal | Items | Calories |
|------|-------|----------|
| Dinner | Stir-Fried Vegetables with Tofu | 300 |

**Dinner:**
- Mixed vegetables (1 cup) - High in vitamins, minerals, and fiber.
- Tofu (1/2 cup) - Provides protein and iron.

### Snacks:
- **Items:** Apple with Almond Butter
- **Calories:** 250

| Meal | Items | Calories |
|------|-------|----------|
| Snack | Apple with Almond Butter | 250 |

**Snack:**
- Fresh apple (1 medium) - Provides fiber and natural sugars.
- Almonds (1/4 cup) - Rich in healthy fats, protein, and minerals.

### Total Calorie Count:
- **Breakfast:** 320
- **Lunch:** 450
- **Dinner:** 300
- **Snack:** 250

**Total Calories for the Day:**
- **1520 calories**

This meal plan is designed to be both nutritious and calorie-controlled, ensuring your patient meets their weight loss goals while maintaining a balanced diet.


⏱️ 5.81s | 1879 chars

💡 Notice how the structured prompt gives a more organized, complete response.


### Exercise 1: Build Your Own Prompt

Pick a task and build a prompt using all 5 components. Try:
- **Scenario A**: A travel itinerary for 2 days in Chennai
- **Scenario B**: A study plan for learning Python in 4 weeks
- **Scenario C**: A product comparison for choosing a laptop

In [95]:
# Fill in your prompt using all 5 components

my_role = ""        # Who should the model be?
my_constraints = "" # What limits or rules?
my_context = ""     # What background info is needed?
my_task = ""        # What should it do?
my_format = ""      # How should it format the response?

if my_task:
    messages = [
        {"role": "system", "content": f"{my_role} {my_constraints}"},
        {"role": "user", "content": f"{my_context} {my_task} {my_format}"}
    ]
    show_messages(messages)
    _ = chat(messages, temperature=0.5)
else:
    print("Fill in the 5 components above, then re-run!")

Fill in the 5 components above, then re-run!


---

## 2. Context Window Management

Every LLM has a **context window** — a maximum number of tokens it can process (input + output combined). Think of it as the model's "working memory."

| Concept | Explanation |
|---------|------------|
| **Token** | A piece of a word. Roughly: 1 token \u2248 4 characters \u2248 \u00be of a word |
| **Context window** | Total tokens the model can handle at once (input + output) |
| **Input tokens** | System prompt + conversation history + user message |
| **Output tokens** | The model's generated response |

```
\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500 Context Window (e.g., 4096 tokens) \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510
\u2502  [System Prompt]  [Conversation History]  [User Msg]  [Response]  \u2502
\u2502  \u2190\u2500\u2500\u2500\u2500\u2500\u2500\u2500 Input Tokens \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2192  \u2190 Output \u2192  \u2502
\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518
```

**Common context window sizes:**

| Model | Context Window |
|-------|---------------|
| GPT-3.5 | 4,096 \u2013 16,384 tokens |
| GPT-4o | 128,000 tokens |
| Claude 3.5 Sonnet | 200,000 tokens |
| Llama 3.2 | 128,000 tokens |
| Qwen 2.5 (1.5B) | 32,768 tokens |

### Experiment 2A: Understanding Token Counts

Let's see how different texts translate into tokens.

In [96]:
import ollama

# Ollama doesn't expose a direct tokenizer, so we estimate:
# Rule of thumb: 1 token ≈ 4 characters (English)

def estimate_tokens(text):
    """Rough token estimate based on character count."""
    return len(text) // 4

texts = [
    ("Short sentence", "Hello, how are you?"),
    ("Medium paragraph", "Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It focuses on developing computer programs that can access data and use it to learn for themselves."),
    ("Long passage", """The history of artificial intelligence dates back to the 1950s when pioneers like Alan Turing and John McCarthy laid the groundwork for what would become one of the most transformative fields in computer science. Turing proposed his famous test in 1950, asking whether machines could think. McCarthy coined the term 'artificial intelligence' at the Dartmouth Conference in 1956. Over the following decades, AI experienced cycles of optimism and disappointment known as 'AI winters.' The field saw renewed interest in the 2010s with the rise of deep learning, fueled by large datasets, powerful GPUs, and advances in neural network architectures. Today, AI powers everything from voice assistants to autonomous vehicles to medical diagnosis systems."""),
]

print(f"{'Text':<20} {'Characters':<12} {'Est. Tokens':<12} {'Words':<8}")
print("-" * 55)
for label, text in texts:
    chars = len(text)
    tokens = estimate_tokens(text)
    words = len(text.split())
    print(f"{label:<20} {chars:<12} {tokens:<12} {words:<8}")

print("\n💡 Rule of thumb: 1 token \u2248 4 characters \u2248 \u00be of a word (in English)")
print("   A 4096-token window \u2248 ~3000 words \u2248 ~6 pages of text")

Text                 Characters   Est. Tokens  Words   
-------------------------------------------------------
Short sentence       19           4            4       
Medium paragraph     250          62           38      
Long passage         748          187          113     

💡 Rule of thumb: 1 token ≈ 4 characters ≈ ¾ of a word (in English)
   A 4096-token window ≈ ~3000 words ≈ ~6 pages of text


### Experiment 2B: What Happens When Context Gets Too Long

When conversation history grows large, the model may:
1. **Forget earlier context** — information at the beginning gets less attention
2. **Produce lower quality responses** — too much noise in the input
3. **Hit the limit and fail** — request gets rejected

Let's simulate a long conversation and test if the model remembers early information.

In [97]:
# Build a conversation where important info is mentioned early,
# then buried under many turns of filler

conversation = [
    {"role": "system", "content": "You are a helpful assistant. Answer questions based on the conversation."},
    # Important info planted at the start
    {"role": "user", "content": "My name is Priya and I'm allergic to peanuts. Remember this."},
    {"role": "assistant", "content": "Got it, Priya! I'll remember that you're allergic to peanuts."},
]

# Add filler conversation turns
filler_topics = [
    ("What's the capital of France?", "The capital of France is Paris."),
    ("Tell me a fun fact about octopuses.", "Octopuses have three hearts and blue blood!"),
    ("What's 17 times 23?", "17 times 23 equals 391."),
    ("Recommend a good book.", "I recommend 'Sapiens' by Yuval Noah Harari."),
    ("What causes rainbows?", "Rainbows are caused by light refracting through water droplets."),
    ("Name 3 programming languages.", "Python, JavaScript, and Rust are popular programming languages."),
    ("Who painted the Mona Lisa?", "Leonardo da Vinci painted the Mona Lisa."),
    ("What's the tallest mountain?", "Mount Everest is the tallest mountain at 8,849 meters."),
    ("Explain photosynthesis briefly.", "Plants convert sunlight, water, and CO2 into glucose and oxygen."),
    ("What year did India gain independence?", "India gained independence in 1947."),
]

for user_msg, asst_msg in filler_topics:
    conversation.append({"role": "user", "content": user_msg})
    conversation.append({"role": "assistant", "content": asst_msg})

# Now ask about the info from the beginning
conversation.append({"role": "user", "content": "What's my name and what am I allergic to?"})

print(f"Total messages in conversation: {len(conversation)}")
total_chars = sum(len(m['content']) for m in conversation)
print(f"Estimated total tokens: ~{total_chars // 4}")
print("\nAsking: 'What's my name and what am I allergic to?'")
print("=" * 60)
_ = chat(conversation, temperature=0.0)

print("\n💡 With a small context, the model should remember.")
print("   But as conversations grow to thousands of tokens, early info can get lost.")

Total messages in conversation: 24
Estimated total tokens: ~242

Asking: 'What's my name and what am I allergic to?'


Your name is Priya and you are allergic to peanuts.


⏱️ 0.59s | 51 chars

💡 With a small context, the model should remember.
   But as conversations grow to thousands of tokens, early info can get lost.


### Experiment 2C: Strategy — Summarize Previous Context

Instead of passing the entire conversation history, **summarize** older turns and keep only recent ones. This saves tokens while preserving important information.

In [98]:
# Strategy: Summarize old context into a compact system message

# Full conversation approach (expensive)
print("=" * 60)
print("APPROACH 1: Full conversation history (23 messages)")
print("=" * 60)

full_conversation = [
    {"role": "system", "content": "You are a project planning assistant."},
    {"role": "user", "content": "I'm building an e-commerce app using React and Node.js."},
    {"role": "assistant", "content": "Great choice! React for the frontend and Node.js for the backend is a popular stack."},
    {"role": "user", "content": "The database will be PostgreSQL."},
    {"role": "assistant", "content": "PostgreSQL is excellent for e-commerce — it handles complex queries and transactions well."},
    {"role": "user", "content": "My team has 3 developers and we have 8 weeks."},
    {"role": "assistant", "content": "With 3 devs and 8 weeks, you'll want to prioritize core features first."},
    {"role": "user", "content": "The budget is $15,000."},
    {"role": "assistant", "content": "That's a reasonable budget for an MVP. Let's focus on essential features."},
    # ... imagine many more turns ...
    {"role": "user", "content": "Give me a sprint plan for the first 2 weeks."}
]

total_chars = sum(len(m['content']) for m in full_conversation)
print(f"Estimated tokens: ~{total_chars // 4}")
_ = chat(full_conversation, temperature=0.3)

APPROACH 1: Full conversation history (23 messages)
Estimated tokens: ~138


Sure! Here’s a rough outline of what you might do in your first two weeks:

### First Week: Planning and Setup

- **Day 1:** Set up PostgreSQL database.
  - Initialize the database with `createdb myecommerce`.
  - Create tables for products, customers, orders, etc.

- **Day 2:** Set up Express.js server using Node.js.
  - Install necessary packages like `express`, `body-parser`, and `mongoose` (for MongoDB).
  
- **Day 3:** Set up React frontend with create-react-app or a similar tool.
  - Create initial components for the home page, product listing, cart, etc.

### Second Week: Core Features

- **Day 1:** Implement user authentication using JWT tokens.
  - Use `passport-jwt` and `jsonwebtoken`.
  
- **Day 2:** Add shopping cart functionality with local storage or a state management library like Redux.
  - Create initial components for adding items to the cart.

- **Day 3:** Integrate payment gateway (e.g., Stripe).
  - Set up an API call to process payments.
  
- **Day 4:** Implement order processing and tracking.
  - Add functionality to create orders, track them in a database, and send notifications.

### Third Week: Testing

- **Day 1:** Test the authentication system thoroughly.
  - Ensure users can log in and out correctly.

- **Day 2:** Test the shopping cart and checkout flow.
  - Verify that adding items to the cart works as expected.

- **Day 3:** Test payment processing.
  - Make sure payments are processed successfully.

### Fourth Week: Final Touches

- **Day 1:** Optimize performance on both frontend and backend.
  - Use tools like `webpack` for bundling, and `eslint`/`prettier` for code formatting.

- **Day 2:** Add additional features based on feedback or user requests (e.g., search functionality).
  
- **Day 3:** Final testing and debugging.
  - Fix any remaining bugs before the final sprint review.

### Fifth Week: Sprint Review

- **Day 1:** Conduct a sprint review meeting with stakeholders to discuss progress, challenges, and next steps.
  
- **Day 2:** Plan for the following week's sprint based on feedback from the review meeting.

This is just a rough outline. You'll need to adjust it based on your specific requirements and team capabilities. Good luck!


⏱️ 5.60s | 2213 chars


In [99]:
# Summarized approach (token-efficient)
print("=" * 60)
print("APPROACH 2: Summarized context (3 messages)")
print("=" * 60)

summarized_conversation = [
    {
        "role": "system",
        "content": (
            "You are a project planning assistant. "
            "Here is a summary of the conversation so far: "
            "The user is building an e-commerce app with React (frontend), "
            "Node.js (backend), and PostgreSQL (database). "
            "Team: 3 developers. Timeline: 8 weeks. Budget: $15,000."
        )
    },
    {"role": "user", "content": "Give me a sprint plan for the first 2 weeks."}
]

total_chars = sum(len(m['content']) for m in summarized_conversation)
print(f"Estimated tokens: ~{total_chars // 4}")
_ = chat(summarized_conversation, temperature=0.3)

print("\n💡 The summarized version uses far fewer tokens but preserves key facts!")

APPROACH 2: Summarized context (3 messages)
Estimated tokens: ~72


Certainly! Here's a rough sprint plan for the first two weeks of your e-commerce app development project:

**Week 1: Planning and Setup**

- **Day 1-3:** Conduct a kickoff meeting with all team members to discuss project goals, scope, timelines, and budget.
- **Day 4-5:** Create an initial project roadmap, including milestones for each week.
- **Day 6-7:** Set up the development environment on both the frontend (React) and backend (Node.js).
- **Day 8:** Install PostgreSQL in a development environment and create a database schema.

**Week 2: Backend Setup**

- **Day 1-3:** Develop the Node.js server using Express. Focus on setting up routes for user authentication, product listing, and cart functionality.
- **Day 4-5:** Implement basic error handling and middleware to ensure smooth interactions between frontend and backend.
- **Day 6-7:** Integrate PostgreSQL into your backend setup by creating a connection pool and defining models for products and users. Start with the initial database schema.

**Week 3: Frontend Setup**

- **Day 1-2:** Set up the React environment on both desktop and mobile platforms using Create React App.
- **Day 3-4:** Develop the user authentication system, including login, registration, and password reset functionalities.
- **Day 5-6:** Implement basic product listing functionality with CRUD operations (Create, Read, Update, Delete).
- **Day 7-8:** Start adding cart functionality to allow users to add products to their shopping carts.

**Week 4: Integration and Testing**

- **Day 1-2:** Integrate the backend API into your frontend application. Focus on making requests for user authentication, product listings, and cart updates.
- **Day 3-4:** Begin testing both frontend and backend functionalities using unit tests and integration tests.
- **Day 5-6:** Perform a thorough code review of all components to ensure they are following best practices and adhering to the project's coding standards.
- **Day 7-8:** Deploy the application to a staging environment for final testing before going live.

**Week 5: Launch Preparation**

- **Day 1-2:** Finalize the frontend UI design, ensuring it aligns with the branding guidelines of your e-commerce app.
- **Day 3-4:** Prepare the backend API documentation and ensure all endpoints are documented clearly.
- **Day 5-6:** Conduct a final review of the project's documentation and any remaining tasks that need to be completed before launch.

**Week 6: Launch**

- **Day 1-2:** Deploy your application to production, ensuring it is accessible via the internet.
- **Day 3-4:** Monitor the initial performance of your app, addressing any bugs or issues immediately.
- **Day 5-6:** Begin collecting user feedback through surveys and reviews. This will help in improving future features.

**Week 7: Post-Launch Support**

- **Day 1-2:** Plan for ongoing support by setting up a system to handle customer queries and technical issues.
- **Day 3-4:** Identify key performance indicators (KPIs) that you want to track over time, such as conversion rates or user engagement metrics.

**Week 8: Review and Next Steps**

- **Day 1-2:** Conduct a project review meeting with all team members to discuss what went well, what could be improved, and how the lessons learned can be applied in future projects.
- **Day 3-4:** Plan for the next sprint or phase of development based on the feedback from this week's review.

This plan is just an outline and should be adjusted according to your specific project requirements and team dynamics.


⏱️ 8.86s | 3519 chars

💡 The summarized version uses far fewer tokens but preserves key facts!


### Experiment 2D: Sliding Window — Keep Only Last N Turns

Another strategy: only keep the **most recent N message pairs** in the conversation. Older messages get dropped.

In [100]:
def sliding_window(messages, max_pairs=3):
    """
    Keep the system message + only the last N user-assistant pairs.
    """
    system_msgs = [m for m in messages if m["role"] == "system"]
    non_system = [m for m in messages if m["role"] != "system"]
    # Keep last max_pairs * 2 messages (each pair = user + assistant)
    trimmed = non_system[-(max_pairs * 2):]
    return system_msgs + trimmed


# Simulate a long conversation
long_conversation = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": "How do I create a list in Python?"},
    {"role": "assistant", "content": "Use square brackets: my_list = [1, 2, 3]"},
    {"role": "user", "content": "How do I add an item?"},
    {"role": "assistant", "content": "Use .append(): my_list.append(4)"},
    {"role": "user", "content": "How do I remove an item?"},
    {"role": "assistant", "content": "Use .remove(value) or .pop(index)"},
    {"role": "user", "content": "How do I sort the list?"},
    {"role": "assistant", "content": "Use .sort() for in-place or sorted() for a new list"},
    {"role": "user", "content": "How do I reverse it?"},
    {"role": "assistant", "content": "Use .reverse() or slicing: my_list[::-1]"},
    # New question
    {"role": "user", "content": "Now how do I find the length?"}
]

print("FULL CONVERSATION")
print(f"Messages: {len(long_conversation)}")
print()

# Apply sliding window
windowed = sliding_window(long_conversation, max_pairs=2)

print("AFTER SLIDING WINDOW (keep last 2 pairs)")
print(f"Messages: {len(windowed)}")
print("-" * 40)
for m in windowed:
    print(f"  [{m['role']:>9}] {m['content'][:60]}")

print(f"\n{'=' * 60}")
print("Response with windowed context:")
_ = chat(windowed, temperature=0.0)

FULL CONVERSATION
Messages: 12

AFTER SLIDING WINDOW (keep last 2 pairs)
Messages: 5
----------------------------------------
  [   system] You are a helpful coding assistant.
  [assistant] Use .sort() for in-place or sorted() for a new list
  [     user] How do I reverse it?
  [assistant] Use .reverse() or slicing: my_list[::-1]
  [     user] Now how do I find the length?

Response with windowed context:


You can use len(my_list) to get the length of the list.


⏱️ 0.34s | 55 chars


### Context Window Management — Summary of Strategies

| Strategy | How It Works | Pros | Cons |
|----------|-------------|------|------|
| **Full history** | Pass everything | Complete context | Expensive, may hit limits |
| **Sliding window** | Keep last N turns | Simple, predictable | Loses old info |
| **Summarization** | Summarize old turns | Compact, preserves key facts | Requires extra LLM call |
| **Hybrid** | Summarize old + keep recent | Best of both | More complex to implement |

### Exercise 2: Token Budget Calculator

Calculate how many conversation turns you can fit in different context windows.

In [101]:
# How many turns fit in a context window?

system_prompt_tokens = 50     # Typical system prompt
avg_user_tokens = 30          # Average user message
avg_assistant_tokens = 150    # Average assistant response
output_reserve = 500          # Tokens reserved for the next response

tokens_per_turn = avg_user_tokens + avg_assistant_tokens  # One turn = user + assistant

context_windows = [4096, 8192, 32768, 128000]

print(f"Assumptions:")
print(f"  System prompt: ~{system_prompt_tokens} tokens")
print(f"  Avg user msg:  ~{avg_user_tokens} tokens")
print(f"  Avg assistant: ~{avg_assistant_tokens} tokens")
print(f"  Output reserve: {output_reserve} tokens")
print(f"  Per turn:      ~{tokens_per_turn} tokens")
print()
print(f"{'Context Window':<18} {'Available':<12} {'Max Turns':<12}")
print("-" * 42)
for window in context_windows:
    available = window - system_prompt_tokens - output_reserve
    max_turns = available // tokens_per_turn
    print(f"{window:>10} tokens  {available:>8}       {max_turns:>5} turns")

print("\n💡 This is why context management matters — even 128K windows fill up in long sessions!")

Assumptions:
  System prompt: ~50 tokens
  Avg user msg:  ~30 tokens
  Avg assistant: ~150 tokens
  Output reserve: 500 tokens
  Per turn:      ~180 tokens

Context Window     Available    Max Turns   
------------------------------------------
      4096 tokens      3546          19 turns
      8192 tokens      7642          42 turns
     32768 tokens     32218         178 turns
    128000 tokens    127450         708 turns

💡 This is why context management matters — even 128K windows fill up in long sessions!


---

## 3. Common Prompt Pitfalls

Even experienced developers write bad prompts sometimes. Here are the most common mistakes and how to fix them.

| Pitfall | Problem | Fix |
|---------|---------|-----|
| **Vague instructions** | Model guesses what you want | Be specific about task, format, and length |
| **Overloaded prompts** | Too many tasks in one prompt | Break into separate prompts |
| **Leading/biased phrasing** | Model follows your bias instead of being objective | Use neutral language |
| **Missing constraints** | Model rambles or adds unwanted content | Set explicit boundaries |
| **Conflicting instructions** | System and user prompts contradict | Align system and user messages |

### Experiment 3A: Vague Instructions \u2192 Inconsistent Output

In [102]:
# Run the same vague prompt 3 times — watch how different the outputs are
print("=" * 60)
print("PITFALL: Vague instructions")
print("=" * 60)

vague_prompt = [{"role": "user", "content": "Write something about dogs."}]

print("Prompt: 'Write something about dogs.'\n")
for run in range(3):
    print(f"--- Run {run + 1} ---")
    result = chat(vague_prompt, temperature=0.8)
    print()

PITFALL: Vague instructions
Prompt: 'Write something about dogs.'

--- Run 1 ---


Dogs have been one of the most popular pets in many cultures for thousands of years due to their loyalty, affectionate nature, and ability to perform tasks such as hunting, guarding, and assisting people with disabilities. Dogs also come in a variety of breeds, each with its own unique characteristics and abilities.
There are two main types of dogs - working dogs and companion dogs. Working dogs include herding dogs, search-and-rescue dogs, police dogs, and military dogs. Companion dogs are those that primarily serve as pets or family members, such as poodles, terriers, and retrievers.
Dogs have a strong sense of smell, which they use to hunt and find their way around the world. They also have exceptional hearing abilities, which allows them to identify sounds in ways humans cannot. Dogs are intelligent animals that can learn tricks and perform tasks with ease.
Overall, dogs make great companions and can be very rewarding pets for those who take the time to properly care for them.


⏱️ 2.28s | 995 chars

--- Run 2 ---


Dogs have been domesticated for thousands of years and are now one of the most popular pets around the world. They come in many breeds with different sizes, colors, and temperaments. Some dogs are gentle and affectionate, while others can be more energetic and adventurous.

The earliest known dog was found in an ancient Egyptian tomb dating back to 3400 BC. Dogs have a strong sense of smell that they use for hunting, protection, and even communication with humans. They also have excellent hearing and vision, which helps them track prey or locate lost items.

Dogs are intelligent animals that can be trained to perform various tasks such as search and rescue, detection of drugs and weapons, and assistance in people's daily lives like guiding the blind and helping those with disabilities. Their loyalty and companionship make them valuable members of many families around the world.


⏱️ 1.96s | 890 chars

--- Run 3 ---


Dogs have been one of the most beloved and versatile pets in history. They are known for their loyalty, intelligence, and ability to form strong bonds with humans. From guarding homes to participating in competitions like dog shows or search and rescue missions, dogs have played an important role in many cultures.

One reason why dogs have captured our hearts is because of their friendly nature. Whether it's wagging its tail or barking excitedly when someone approaches, a dog can instantly let you know they are happy to be around people. This makes them great companions and perfect pets for families with children, as well as single individuals looking for a furry friend.

In addition to being beloved by humans, dogs have also been the subject of much scientific research due to their unique ability to communicate using vocalizations, body language, and facial expressions. Scientists are still learning about all the things that make these animals so interesting and fascinating.

Overall, whether it's as a pet or working dog, there is something for everyone when it comes to dogs. Their friendly nature, intelligence, and adaptability have made them one of the most beloved pets in the world.


⏱️ 2.53s | 1205 chars



In [103]:
# --- Fixed version ---
print("=" * 60)
print("FIXED: Specific instructions")
print("=" * 60)

specific_prompt = [
    {"role": "user", "content": (
        "Write 3 fun facts about golden retrievers. "
        "Use a numbered list. One sentence per fact."
    )}
]

print("Prompt: 'Write 3 fun facts about golden retrievers...'\n")
for run in range(3):
    print(f"--- Run {run + 1} ---")
    result = chat(specific_prompt, temperature=0.8)
    print()

print("💡 Much more consistent! Specificity reduces variance.")

FIXED: Specific instructions
Prompt: 'Write 3 fun facts about golden retrievers...'

--- Run 1 ---


Sure! Here are three fun facts about Golden Retrievers:

1. **Golden Retrievers Have Great Memories**: These dogs are renowned for their excellent memory, which helps them to remember names and faces well. This is often cited as one of the main reasons why they excel in retrieving tasks such as hunting or assisting with water rescue.

2. **Golden Retrievers Are Good Watch Dogs**: While not bred specifically as guard dogs due to being more affectionate, Golden Retrievers can be quite good at keeping an eye on their surroundings and alerting owners to unusual activity around the home or outside yard.

3. **Golden Retrievers Need Plenty of Exercise**: Like many large breeds, they require a lot of physical exertion to maintain overall health. This means regular walks and vigorous playtime in suitable outdoor environments are crucial for Golden Retrievers' well-being.


⏱️ 2.01s | 875 chars

--- Run 2 ---


1. Golden Retrievers have a double coat consisting of a soft undercoat and a longer, coarser outer layer.
2. They were originally bred to be hunting dogs but are now commonly used as family pets due to their gentle disposition and friendly nature.
3. Golden Retrievers can live up to 10 years or more with proper care, making them long-lived companions for many owners.


⏱️ 0.87s | 369 chars

--- Run 3 ---


1. Golden Retrievers have a unique ability to recognize and remember the scents of their owners, often sniffing out familiar smells for hours.
2. They are highly intelligent dogs known for being friendly, obedient, and trainable. Their gentle temperament makes them excellent companions.
3. These dogs have an average lifespan of 10-14 years, which is longer than some other breeds but shorter than others like the Great Dane or Saint Bernard.


⏱️ 1.03s | 443 chars

💡 Much more consistent! Specificity reduces variance.


### Experiment 3B: Overloaded Prompts \u2192 Incomplete Output

Asking the model to do too many things in one prompt often leads to some tasks being done poorly or skipped.

In [104]:
# --- Overloaded: 5 tasks crammed into one prompt ---
print("=" * 60)
print("PITFALL: Overloaded prompt (too many tasks)")
print("=" * 60)

overloaded = [
    {"role": "user", "content": (
        "Explain what a REST API is, give me a Python code example, "
        "list the HTTP methods, compare REST vs GraphQL, "
        "and suggest when to use each one."
    )}
]
show_messages(overloaded)
_ = chat(overloaded, temperature=0.3, num_predict=300)

print("\n💡 Notice: With a token limit, some tasks get skipped or rushed.")

PITFALL: Overloaded prompt (too many tasks)


### What is a REST API?

A REST (Representational State Transfer) API is a type of web application programming interface that allows clients to interact with a server using standard HTTP methods such as GET, POST, PUT, DELETE, etc. It's designed to be stateless and follows the principles of uniform resource identification (URI), caching, and data transfer via hypermedia.

### Python Code Example

Here is an example of a simple REST API in Python using Flask:

```python
from flask import Flask, jsonify, request

app = Flask(__name__)

# Sample database for demonstration purposes
users = [
    {"id": 1, "username": "user1"},
    {"id": 2, "username": "user2"}
]

@app.route('/api/v1/users', methods=['GET'])
def get_users():
    return jsonify(users)

@app.route('/api/v1/users/<int:user_id>', methods=['GET'])
def get_user(user_id):
    user = next((u for u in users if u['id'] == user_id), None)
    if user is not None:
        return jsonify(user)
    else:
        return {"message": "User not found"}, 404

@app.route('/api/v1/users', methods=['POST'])
def create_user():
    new_user = request.get_json()
    users.append(new_user)
    return jsonify(users[-1]), 201

if __name__ == '__main__':
    app.run(debug=True


⏱️ 3.48s | 1230 chars

💡 Notice: With a token limit, some tasks get skipped or rushed.


In [105]:
# --- Fixed: Break into focused prompts ---
print("=" * 60)
print("FIXED: One task at a time")
print("=" * 60)

focused_tasks = [
    "Explain what a REST API is in 2-3 sentences.",
    "List the 5 main HTTP methods with a one-line description each.",
    "Compare REST vs GraphQL in a 3-row table (columns: Feature, REST, GraphQL).",
]

for i, task in enumerate(focused_tasks, 1):
    print(f"\n--- Task {i} ---")
    messages = [{"role": "user", "content": task}]
    _ = chat(messages, temperature=0.3)

print("\n💡 Each response is now complete and focused!")

FIXED: One task at a time

--- Task 1 ---


A REST (Representational State Transfer) API is a type of application programming interface that allows for the communication between client and server using HTTP requests. It follows principles such as statelessness, caching, and uniform interfaces to enable efficient and scalable interactions over the internet.


⏱️ 0.62s | 314 chars

--- Task 2 ---


1. GET: Retrieve data from the server.
2. POST: Send data to the server and create new resources.
3. PUT/PATCH: Replace or update an existing resource on the server.
4. DELETE: Remove a resource from the server permanently.
5. HEAD: Retrieve only the headers of a response, without the body.


⏱️ 0.85s | 291 chars

--- Task 3 ---


Certainly! Below is a comparison of REST and GraphQL in three rows:

| **Feature** | **REST** | **GraphQL** |
|-------------|----------|--------------|
| **Documentation** | Yes | No |
| **Error Handling** | Built-in error handling with HTTP status codes | Error messages are specific to the API, not tied to HTTP status codes |
| **Scalability** | Scalable through load balancing and caching mechanisms | Can be more scalable due to its ability to handle multiple requests simultaneously without a single point of failure |
| **Performance** | Generally slower due to JSON serialization/deserialization overhead | Faster performance as it can use native data types, reducing the need for conversion |
| **Flexibility** | Limited by the number and type of HTTP methods (GET, POST, PUT, DELETE) | Highly flexible with a wide range of queries and mutations |
| **Client-Side Complexity** | Generally simpler to implement on client-side due to fewer HTTP requests required per request | More complex for clients as it requires more interaction between the server and client |
| **State Management** | State is often managed by the client, which can lead to state inconsistencies | Can manage state directly in the database or through mutations, reducing the need for client-side state management |
| **Data Fetching** | Typically involves multiple HTTP requests per request | Single query typically fetches all necessary data at once |
| **Testing** | Easier with mocking and stubbing libraries | More difficult due to its dynamic nature |

This table provides a quick overview of how RESTful APIs and GraphQL differ in their design, implementation, and use cases.


⏱️ 3.70s | 1661 chars

💡 Each response is now complete and focused!


### Experiment 3C: Leading / Biased Prompts

The way you phrase a question can **bias** the model's response. This is especially dangerous for tasks requiring objectivity.

In [106]:
system = {"role": "system", "content": "You are an objective technology analyst. Give balanced assessments."}

prompts = [
    {
        "label": "BIASED (leading question)",
        "content": "Why is Python the best programming language for everything?"
    },
    {
        "label": "NEUTRAL (balanced question)",
        "content": "What are the strengths and weaknesses of Python compared to other languages?"
    }
]

for p in prompts:
    print(f"\n{'=' * 60}")
    print(f"  {p['label']}")
    print(f"{'=' * 60}")
    messages = [system, {"role": "user", "content": p["content"]}]
    show_messages(messages)
    _ = chat(messages, temperature=0.3, num_predict=250)

print("\n💡 The biased prompt pushes the model to agree. Neutral phrasing gets balanced analysis.")


  BIASED (leading question)


Python is often considered the "best" programming language due to several factors that make it highly versatile and beneficial in various domains:

1. **Ease of Learning**: Python has a relatively simple syntax, making it accessible even for beginners. Its readability and simplicity allow developers to write code quickly without being bogged down by complex syntax rules.

2. **Rich Standard Library**: Python comes with an extensive standard library that includes modules for everything from web development (Flask/Django), data analysis (NumPy/Pandas), machine learning (Scikit-learn/Keras), networking, and more.

3. **Community Support**: The Python community is incredibly supportive of its users. There are numerous resources available online including forums, documentation, and libraries that can help solve problems or find solutions to issues developers face.

4. **Versatility in Applications**: Python is used across a wide range of applications from web development (Django/Flask), scientific computing (NumPy/Pandas), artificial intelligence (TensorFlow/Keras), data analysis, automation scripts, mobile app development, and more.

5. **Dynamic Typing**: Python supports dynamic typing which means types are determined at runtime rather than compile time, making it easier to write code that is both readable and maintain


⏱️ 2.96s | 1337 chars

  NEUTRAL (balanced question)


Python is a versatile programming language that has several strengths and some notable weaknesses when compared to other popular languages like Java, C++, or JavaScript.

### Strengths:

1. **Ease of Learning:**
   - **Strength:** Python's syntax is designed for readability and simplicity, making it easier for beginners to pick up.
   - **Weakness:** While this can be a strength, it might also mean that advanced features and performance-critical applications may require more effort from experienced developers.

2. **Versatility:**
   - **Strength:** Python can be used in a wide range of applications, including web development (with frameworks like Django or Flask), scientific computing (with libraries like NumPy and SciPy), data analysis, artificial intelligence, machine learning, automation scripts, and more.
   - **Weakness:** It might not be the best choice for systems programming where low-level operations are crucial.

3. **Community Support:**
   - **Strength:** Python has a large and active community that provides extensive documentation, libraries, and support through forums like Stack Overflow.
   - **Weakness:** This can sometimes lead to "Pythonitis," where developers become overly reliant on the language's features without considering other options.

4. **Performance:


⏱️ 2.98s | 1300 chars

💡 The biased prompt pushes the model to agree. Neutral phrasing gets balanced analysis.


### Experiment 3D: Missing Constraints \u2192 Model Rambles

In [107]:
# --- No constraints: model decides everything ---
print("=" * 60)
print("PITFALL: No constraints")
print("=" * 60)

no_constraints = [{"role": "user", "content": "Tell me about climate change."}]
result_nc = chat(no_constraints, temperature=0.5)
print(f"\n  [Word count: {len(result_nc.split())}]")

PITFALL: No constraints


Climate change refers to the long-term changes in Earth's weather patterns and temperature over time. These changes can be caused by various factors such as natural phenomena like volcanic eruptions or solar activity, but most of the current warming is due to human activities that release greenhouse gases into the atmosphere.

The primary cause of climate change today is the increase in carbon dioxide (CO2) emissions from burning fossil fuels for energy and transportation. This leads to a phenomenon known as the Greenhouse Effect, where certain gases trap heat in the Earth's atmosphere, leading to global warming and changes in weather patterns.

Climate change has several impacts on our planet:

1. Rising sea levels: As ice caps melt due to warmer temperatures, more water is added to oceans, causing them to rise. This threatens coastal cities and low-lying islands.
2. More extreme weather events: Warmer air can hold more moisture, leading to increased intensity of storms like hurricanes and typhoons. It also contributes to droughts in some regions.
3. Changes in ecosystems: Many species are shifting their ranges poleward or elevating higher as temperatures rise, which can disrupt food chains and alter habitats for various organisms.

The effects of climate change have been observed around the world. For instance:

- The Arctic is warming twice as fast as lower latitudes, leading to dramatic changes in permafrost thawing.
- Coral reefs are bleaching more frequently due to higher water temperatures, threatening marine life that relies on them.
- Many species are moving towards cooler areas where they can survive at the current temperature.

Addressing climate change requires global cooperation and significant reductions in greenhouse gas emissions. This includes transitioning to renewable energy sources like wind and solar power, improving energy efficiency, and implementing policies to reduce deforestation.


⏱️ 4.19s | 1939 chars

  [Word count: 296]


In [108]:
# --- With constraints: focused and controlled ---
print("=" * 60)
print("FIXED: Clear constraints")
print("=" * 60)

with_constraints = [
    {
        "role": "system",
        "content": "You are a science communicator. Keep responses concise and factual."
    },
    {
        "role": "user",
        "content": (
            "Explain the top 3 causes of climate change. "
            "Use a numbered list. One sentence per cause. "
            "No more than 50 words total."
        )
    }
]
result_wc = chat(with_constraints, temperature=0.3)
print(f"\n  [Word count: {len(result_wc.split())}]")

print(f"\n💡 Unconstrained: {len(result_nc.split())} words vs. Constrained: {len(result_wc.split())} words")

FIXED: Clear constraints


1. Increased greenhouse gases trap heat in Earth's atmosphere, leading to global warming.
2. Deforestation reduces the planet's ability to absorb carbon dioxide through photosynthesis.
3. Natural processes like volcanic eruptions also contribute to climate change by releasing large amounts of greenhouse gases into the atmosphere.


⏱️ 0.71s | 331 chars

  [Word count: 46]

💡 Unconstrained: 296 words vs. Constrained: 46 words


### Experiment 3E: Conflicting Instructions

What happens when the system prompt says one thing and the user prompt says another?

In [109]:
# --- Conflicting: system says formal, user says casual ---
print("=" * 60)
print("PITFALL: Conflicting instructions")
print("=" * 60)

conflicting = [
    {
        "role": "system",
        "content": "You are a formal academic professor. Always use scholarly language and cite sources."
    },
    {
        "role": "user",
        "content": "Hey bro, explain quantum physics like I'm 5. Keep it super chill and funny. Use slang."
    }
]
show_messages(conflicting)
_ = chat(conflicting, temperature=0.5)

print("\n💡 The model is confused \u2014 it tries to satisfy both, resulting in an awkward mix.")

PITFALL: Conflicting instructions


Sure thing! So, you know how sometimes when you're playing with your toys, they act up? Well, imagine if those toys could talk to each other and do tricks without any batteries or wires. That’s kind of like what quantum physics is all about!

Now, let's make it a bit more fun:

Imagine you have a bunch of invisible balloons floating around in the air. These balloons are so small that you can't see them with your eyes. But if they were super strong and could carry tiny objects, they would be really cool! Now, these balloons don’t just float up; they also spin around like crazy!

In quantum physics, we use something called "particles" to help us understand how things work at the tiniest level. These particles are so small that if you tried to touch one with your hand, it wouldn't even feel warm! But guess what? They can still do tricks and play games without anyone telling them to!

So remember: quantum physics is all about these invisible balloons (particles) that can talk to each other and do amazing things in the tiniest of spaces. It’s like a whole new world where everything works differently, but it's super fun!


⏱️ 2.89s | 1132 chars

💡 The model is confused — it tries to satisfy both, resulting in an awkward mix.


In [110]:
# --- Fixed: aligned instructions ---
print("=" * 60)
print("FIXED: Aligned instructions")
print("=" * 60)

aligned = [
    {
        "role": "system",
        "content": "You are a fun, casual science explainer. Use simple language and humor."
    },
    {
        "role": "user",
        "content": "Explain quantum physics like I'm 5. Keep it super chill and funny."
    }
]
show_messages(aligned)
_ = chat(aligned, temperature=0.5)

print("\n💡 When system and user are aligned, the output is coherent and on-tone.")

FIXED: Aligned instructions


Sure thing! Let's talk about the weird world of tiny things that go bump in the night.

Imagine you have a magical box. When you open it, inside there’s a ball with a light on it. The light is special because it can be red or blue. Now, imagine we could change how bright the light is by blowing air over it really fast. 

Now, here's where it gets super cool: We don't know exactly what color the light will be when we open the box! It’s a bit like magic because we can’t predict for sure. But that’s okay because it’s still fun and makes our world more magical!

So remember, even though we can describe things in math and science, there are some parts of nature that just don't work the same way as the stuff you see every day. Isn’t that cool? It's like a whole new universe hiding inside everything!


⏱️ 2.25s | 804 chars

💡 When system and user are aligned, the output is coherent and on-tone.


### Exercise 3: Fix the Broken Prompts

Each prompt below has a problem. Run it, identify the issue, then fix it in the cell after.

In [111]:
# --- Broken Prompt 1: Too vague ---
print("BROKEN PROMPT 1")
print("=" * 40)
broken_1 = [{"role": "user", "content": "Help me with my project."}]
_ = chat(broken_1)

BROKEN PROMPT 1


Of course! I'm here to help you with your project. Please feel free to ask me any specific questions or share what you've already done so far. The more details you can provide, the better I can assist you.


⏱️ 0.65s | 205 chars


In [112]:
# Fix broken prompt 1 here:
fixed_1 = [{"role": "user", "content": ""}]  # <-- Write your fix

if fixed_1[0]["content"]:
    print("YOUR FIX:")
    _ = chat(fixed_1)
else:
    print("Write your improved prompt above!")

Write your improved prompt above!


In [113]:
# --- Broken Prompt 2: Conflicting tone ---
print("BROKEN PROMPT 2")
print("=" * 40)
broken_2 = [
    {"role": "system", "content": "You are a children's storyteller. Use fun, playful language."},
    {"role": "user", "content": "Write a technical analysis of TCP/IP networking protocols with RFC references."}
]
_ = chat(broken_2)

BROKEN PROMPT 2


Hey there, little network explorer! Today, we're going to dive into the world of the Internet – oh, so big and magical! We'll be talking about something called TCP/IP, which helps all your gadgets talk to each other across miles.

Imagine you have a bunch of toys (like computers) that need to share their toys with each other. TCP/IP is like the magic code they use to communicate without getting into trouble!

Let's break it down:

1. **TCP – The Traffic Cop**:
   Think of TCP as a traffic cop. It makes sure all the little devices talk nicely and don’t cause any accidents on the internet road. Without TCP, your toys might crash when you try to share something with others.

2. **IP – The Internet Packet**:
   IP is like the packet that holds everything your toy needs to travel across the whole world. It’s so good at its job that it can even find a way for your toy to reach a faraway friend! IP has this special ability because of what we call "RFCs."

3. **RFCs – The Reference Guidelines**:
   RFC stands for Request For Comments, which are like blueprints that help the world understand how these two magic codes work together so they don’t make any mistakes. They’re super important to keep everyone’s toys safe and happy on their journey across the internet!

So remember, next time you're playing with your friends online or sending a message to grandma, just think about TCP/IP making sure it all works smoothly, thanks to RFCs giving them clear instructions!


⏱️ 3.91s | 1476 chars


In [114]:
# Fix broken prompt 2 here:
fixed_2 = [
    {"role": "system", "content": ""},  # <-- Fix the system prompt
    {"role": "user", "content": ""}      # <-- Fix the user prompt
]

if fixed_2[0]["content"] and fixed_2[1]["content"]:
    print("YOUR FIX:")
    _ = chat(fixed_2)
else:
    print("Write your improved prompts above!")

Write your improved prompts above!


In [115]:
# --- Broken Prompt 3: No format, no constraints ---
print("BROKEN PROMPT 3")
print("=" * 40)
broken_3 = [{"role": "user", "content": "Compare React and Angular."}]
_ = chat(broken_3)

BROKEN PROMPT 3


React and Angular are both popular front-end JavaScript frameworks used for building user interfaces in web applications. While they serve similar purposes, there are some key differences between them:

1. Architecture:
- React is a component-based framework that follows the "Single Responsibility" principle.
- Angular follows an MVC (Model-View-Controller) architecture.

2. Development Process:
- In React, developers typically work on individual components, which can be composed to form more complex applications.
- With Angular, it's common to begin with full-blown projects and then refactor or expand as needed.

3. Performance Optimization:
- React focuses on improving performance by minimizing unnecessary re-renders through smart updates.
- Angular employs TypeScript and Ahead-of-Time (AOT) compilation for better performance.

4. Ecosystem and Community:
- React has a larger, more active community and is well-known for its accessibility features in the browser.
- Angular is less common but can be used to create highly complex enterprise applications with mature infrastructure support.

5. Learning Curve:
- Both frameworks have steep learning curves due to their complexity but offer different paths based on individual use cases.

6. Features:
- React is more lightweight and easier to start with, focusing on simplicity.
- Angular offers richer features like two-way data binding, dependency injection, and an extensive set of built-in services.

7. Development Tools:
- Both frameworks integrate well with popular development tools like Webpack for bundling and ESLint for linting code quality.

8. Compatibility:
- React is primarily used in a single-page application (SPA) environment.
- Angular is more suited to multi-page applications that require the integration of other technologies like Angular Material, AOT compilation, and TypeScript.

9. Ecosystem Integration:
- Angular integrates well with Spring Boot and Node.js thanks to their interoperability layers.
- React can work seamlessly with various libraries and front-end tools.

10. Flexibility:
- Both frameworks offer high flexibility in terms of component composition and custom styling possibilities.

While both are powerful solutions, the choice between them depends on specific project requirements such as performance needs, team familiarity, or existing infrastructure setup.


⏱️ 5.18s | 2371 chars


In [116]:
# Fix broken prompt 3 here:
fixed_3 = [{"role": "user", "content": ""}]  # <-- Add format and constraints

if fixed_3[0]["content"]:
    print("YOUR FIX:")
    _ = chat(fixed_3)
else:
    print("Write your improved prompt above!")

Write your improved prompt above!


---

## Key Takeaways

### Prompt Anatomy

| Component | Purpose | Impact on Output |
|-----------|---------|------------------|
| **Task** | What to do | Determines the core response |
| **Context** | Background info | Makes response relevant |
| **Role** | Who to be | Shapes tone and expertise level |
| **Format** | How to structure | Controls output layout |
| **Constraints** | What limits to follow | Prevents rambling and off-topic content |

### Context Window Management

| Strategy | When to Use |
|----------|-------------|
| **Full history** | Short conversations (< 10 turns) |
| **Sliding window** | Chatbots where recent context matters most |
| **Summarization** | Long sessions where early facts are important |
| **Token budgeting** | Production systems with cost constraints |

### Common Pitfalls Cheat Sheet

| Pitfall | Quick Fix |
|---------|----------|
| Vague instructions | Add task + format + constraints |
| Overloaded prompt | One task per prompt |
| Biased/leading phrasing | Use neutral "what are the pros and cons" |
| No constraints | Set length, format, and scope limits |
| Conflicting instructions | Align system and user prompts |

---

*Nunnari Academy | Generative AI & Agentic AI Professional Program*  
*Module 1, Week 2, Section 2.1 | Prompt Anatomy, Context Windows & Common Pitfalls*